In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('schizophrenia_dataset.csv')
df.head()  # Shows first 5 rows

,Patient_ID,Age,Gender,Education_Level,Marital_Status,Occupation,Income_Level,Living_Area,Diagnosis,Disease_Duration,Hospitalizations,Family_History,Substance_use,Suicide_Attempt,Positive_Symptom_Score,Negative_Symptom_Score,GAF_Score,Social_Support,Stress_Factors,Medication_Adherence
0,1,72,1,4,2,0,2,1,0,0,0,0,0,0,32,48,72,0,2,2
1,2,49,1,5,2,2,1,0,1,35,1,1,1,1,51,63,40,2,2,0
2,3,53,1,5,3,2,1,0,1,32,0,1,0,0,72,85,51,0,1,1
3,4,67,1,3,2,0,2,0,0,0,0,0,1,0,10,21,74,1,1,2
4,5,54,0,1,2,0,2,1,0,0,0,0,0,0,4,27,98,0,1,0


In [5]:
print("Shape:", df.shape)
print("\nColumns:", df.columns.tolist())
df.info()

Shape: (10000, 20)

Columns: ['Patient_ID', 'Age', 'Gender', 'Education_Level', 'Marital_Status', 'Occupation', 'Income_Level', 'Living_Area', 'Diagnosis', 'Disease_Duration', 'Hospitalizations', 'Family_History', 'Substance_use', 'Suicide_Attempt', 'Positive_Symptom_Score', 'Negative_Symptom_Score', 'GAF_Score', 'Social_Support', 'Stress_Factors', 'Medication_Adherence']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   Patient_ID              10000 non-null  int64
 1   Age                     10000 non-null  int64
 2   Gender                  10000 non-null  int64
 3   Education_Level         10000 non-null  int64
 4   Marital_Status          10000 non-null  int64
 5   Occupation              10000 non-null  int64
 6   Income_Level            10000 non-null  int64
 7   Living_Area             10000 non-null  int64
 8   Di

In [7]:
df.isnull().sum()

Patient_ID                0
Age                       0
Gender                    0
Education_Level           0
Marital_Status            0
Occupation                0
Income_Level              0
Living_Area               0
Diagnosis                 0
Disease_Duration          0
Hospitalizations          0
Family_History            0
Substance_use             0
Suicide_Attempt           0
Positive_Symptom_Score    0
Negative_Symptom_Score    0
GAF_Score                 0
Social_Support            0
Stress_Factors            0
Medication_Adherence      0
dtype: int64

In [9]:
df = df.drop(columns=['Patient_ID'])

In [11]:
'Patient_ID' in df.columns

False

In [13]:
# 1. One-hot encode nominal (unordered) columns
# These columns are: Marital_Status, Occupation, Living_Area
df = pd.get_dummies(df, columns=['Marital_Status', 'Occupation', 'Living_Area'])

# 2. Ensure ordinal and binary columns are integer type
ordinal_binary_cols = [
    'Education_Level', 'Income_Level', 'Social_Support', 'Stress_Factors',
    'Medication_Adherence', 'Gender', 'Family_History', 'Substance_use', 'Suicide_Attempt'
]
for col in ordinal_binary_cols:
    df[col] = df[col].astype(int)

# View the first few rows to confirm changes
df.head()

,Age,Gender,Education_Level,Income_Level,Diagnosis,Disease_Duration,Hospitalizations,Family_History,Substance_use,Suicide_Attempt,...,Marital_Status_0,Marital_Status_1,Marital_Status_2,Marital_Status_3,Occupation_0,Occupation_1,Occupation_2,Occupation_3,Living_Area_0,Living_Area_1
0,72,1,4,2,0,0,0,0,0,0,...,False,False,True,False,True,False,False,False,False,True
1,49,1,5,1,1,35,1,1,1,1,...,False,False,True,False,False,False,True,False,True,False
2,53,1,5,1,1,32,0,1,0,0,...,False,False,False,True,False,False,True,False,True,False
3,67,1,3,2,0,0,0,0,1,0,...,False,False,True,False,True,False,False,False,True,False
4,54,0,1,2,0,0,0,0,0,0,...,False,False,True,False,True,False,False,False,False,True


In [15]:
from sklearn.model_selection import train_test_split

# 1. Set features (X) and target (y)
X = df.drop(columns=['Diagnosis'])
y = df['Diagnosis']

# 2. Train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Show shapes to confirm
print("Train shape:", X_train.shape, y_train.shape)
print("Test shape:", X_test.shape, y_test.shape)

Train shape: (8000, 25) (8000,)
Test shape: (2000, 25) (2000,)


In [17]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [19]:
print(X_train_scaled.shape)
print(X_test_scaled.shape)

(8000, 25)
(2000, 25)


In [21]:
from imblearn.over_sampling import SMOTE

# Initialize SMOTE
smote = SMOTE(random_state=42)

# Apply SMOTE only on the training data
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)

# Check the new class distribution
from collections import Counter
print("Before SMOTE:", Counter(y_train))
print("After SMOTE:", Counter(y_train_balanced))

Before SMOTE: Counter({0: 5690, 1: 2310})
After SMOTE: Counter({0: 5690, 1: 5690})


C:\Users\Haseeb\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\Haseeb\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\Haseeb\anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Haseeb\anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\Haseeb\anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.Crea

In [25]:
from sklearn.feature_selection import SelectKBest, f_classif

# Example: select top 10 features
selector = SelectKBest(score_func=f_classif, k=10)
X_train_selected = selector.fit_transform(X_train_balanced, y_train_balanced)
X_test_selected = selector.transform(X_test_scaled)

In [27]:
import pandas as pd

# Optionally, use feature names if you have them
feature_names = X.columns[selector.get_support()]  # if X is a DataFrame

X_train_df = pd.DataFrame(X_train_selected, columns=feature_names)
y_train_df = pd.DataFrame(y_train_balanced, columns=['Diagnosis'])
X_train_df.to_csv('X_train_selected.csv', index=False)
y_train_df.to_csv('y_train_balanced.csv', index=False)